In [1]:
import movingpandas as mp # needed to delete line calling contextily from trajectory.py
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
import geopandas as gpd
from shapely.geometry import Point, Polygon
import warnings
import matplotlib.pyplot as plt
import descartes
import pyproj

plt.rcParams['axes.xmargin'] = 0.1
plt.rcParams['axes.ymargin'] = 0.1
%matplotlib inline 
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Reading the data

In [2]:
gps = pd.read_csv('./sample_data/traj_full.csv')
gps.head()

,trID,trN,pIdx,X,Y,time,duration
0,a,1,1,7.213,50.772,04/12/2006 19:14:41,0
1,a,1,2,7.213,50.772,04/12/2006 19:14:45,0
2,a,1,3,7.212,50.772,04/12/2006 19:14:51,0
3,a,1,4,7.212,50.772,04/12/2006 19:14:55,0
4,a,1,5,7.211,50.771,04/12/2006 19:15:00,0


In [3]:
gps = gps.sort_values(by=['trID', 'time'])
gps['date_time'] = pd.to_datetime(gps['time'])
gps.shape

(112537, 8)

In [4]:
# I created three user IDS cause I would like to have a linestring per user
ids = np.array(["A", "B", "C"])
gps['user_id'] = np.repeat(ids, [37000, 37000, 38537], axis=0)

In [5]:
trips = gps.set_index('date_time')
lon_lat = [Point(xy) for xy in zip(trips.X, trips.Y)]
crs = {'init': 'epsg:4326'}
trips = gpd.GeoDataFrame(trips, crs=crs, geometry=lon_lat)
trips.head()

,trID,trN,pIdx,X,Y,time,duration,user_id,geometry
date_time,,,,,,,,,
2007-01-01 14:40:36,a,1,8745,7.203,50.780,01/01/2007 14:40:36,0,A,POINT (7.202585000000001 50.779636)
2007-01-01 14:40:39,a,1,8746,7.203,50.780,01/01/2007 14:40:39,0,A,POINT (7.20273 50.779854)
2007-01-01 14:40:43,a,1,8747,7.203,50.780,01/01/2007 14:40:43,0,A,POINT (7.202917 50.78012)
2007-01-01 14:40:46,a,1,8748,7.203,50.780,01/01/2007 14:40:46,0,A,POINT (7.203072 50.78035999999999)
2007-01-01 14:40:49,a,1,8749,7.203,50.781,01/01/2007 14:40:49,0,A,POINT (7.203155000000001 50.78064000000001)


In [6]:
# to make everything faster I just selected the first 300 observations for each user
trips = trips.groupby('user_id').head(300)
trips.head()

,trID,trN,pIdx,X,Y,time,duration,user_id,geometry
date_time,,,,,,,,,
2007-01-01 14:40:36,a,1,8745,7.203,50.780,01/01/2007 14:40:36,0,A,POINT (7.202585000000001 50.779636)
2007-01-01 14:40:39,a,1,8746,7.203,50.780,01/01/2007 14:40:39,0,A,POINT (7.20273 50.779854)
2007-01-01 14:40:43,a,1,8747,7.203,50.780,01/01/2007 14:40:43,0,A,POINT (7.202917 50.78012)
2007-01-01 14:40:46,a,1,8748,7.203,50.780,01/01/2007 14:40:46,0,A,POINT (7.203072 50.78035999999999)
2007-01-01 14:40:49,a,1,8749,7.203,50.781,01/01/2007 14:40:49,0,A,POINT (7.203155000000001 50.78064000000001)


## Create Linestrings

In [7]:
t_start = datetime.now()
trajectories = []
for key, values in trips.groupby(['user_id']):
    trajectory = mp.Trajectory(key, values)
    trajectories.append(trajectory)
    print("Finished creating {} trajectories in {}".format(len(trajectories), datetime.now() - t_start))  

Finished creating 1 trajectories in 0:00:00
Finished creating 2 trajectories in 0:00:00
Finished creating 3 trajectories in 0:00:00.022657


In [8]:
geod = pyproj.Geod(ellps='WGS84')

## Extract charactheristics points

In [19]:
# Extract charactheristic points
# In Extract characteristic trajectory points, distance parameters are specified in meters, stop duration in seconds, 
# and angles in degrees. The characteristic points contain start and end locations, as well as turns and stop locations:
# In Extract characteristic trajectory points, distance parameters are specified in meters, stop duration in seconds, 
# and angles in degrees. The characteristic points contain start and end locations, as well as turns and stop locations:
class Analyzer:
    def __init__(self,traj, max_distance, min_distance, min_stop_duration, min_angle = 45):
        self.i = 0
        self.j = 1
        self.k = 2
        self.traj = traj
        self.n = self.traj.df.geometry.count()
        self.max_distance = max_distance
        self.min_distance = min_distance
        self.min_stop_duration = min_stop_duration
        self.min_angle = min_angle
        self.last_point_index = int(traj.df.geometry.count() - 1)
        # or self.n - 1
        self.significant_points = [self.traj.to_linestring().coords[0],self.traj.to_linestring().coords[self.last_point_index]]
        #self.d = QgsDistanceArea()
        #self.d.setEllipsoid('WGS84')
        #self.d.setEllipsoidalMode(True)
        
    def find_significant_points(self):
        
        while self.j < self.n-1:
            
            #print(self.j)
            
            if self.is_representative_max_distance():
                #print("representative max distance at {0}".format(self.j))
                self.significant_points.append(self.traj.to_linestring().coords[int(self.j)])
                self.i = self.j
                self.j = self.i + 1
                continue
                
            elif self.more_points_further_than_min_distance():
                if self.k > self.j + 1: # trajectories[5].df.index[0]
                    d_time = (self.traj.df.index[int(self.k-1)] - self.traj.df.index[int(self.j)]).total_seconds()
                    if d_time >= self.min_stop_duration: # step 7 in andrienkos paper!
                        #print("significant stop ({1}) at {0}".format(self.j,d_time))
                        self.significant_points.append(self.traj.to_linestring().coords[int(self.j)])
                        self.i = self.j
                        self.j = self.k
                        continue 
                    else:
                        # compute the average spatial position to represent the similar points
                        m = self.j + (self.k-1-self.j)//2 # had to use // instead of /
                        self.j = m # what is this m for?
                
                a_turn = self.compute_angle_between_vectors()
                #print("{0}: {1}".format(self.j,a_turn))
                if a_turn >= self.min_angle and a_turn <= (360-self.min_angle):
                    #print("significant angle ({0}) at {1}".format(a_turn,self.j))
                    self.significant_points.append(self.traj.to_linestring().coords[int(self.j)])
                    self.i = self.j
                    self.j = self.k
                else:
                    self.j += 1
                        
            else:
                return self.significant_points
        
        return self.significant_points
   
    def compute_angle_between_vectors(self):
        p_i = self.traj.to_linestring().coords[int(self.i)]
        p_j = self.traj.to_linestring().coords[int(self.j)]
        p_k = self.traj.to_linestring().coords[int(self.k)]
        azimuth_ij = geod.inv(p_i[0], p_i[1], p_j[0], p_j[1])[0] # in position 0 is azimuth 1 in position 2 is the distance
        azimuth_jk = geod.inv(p_j[0], p_j[1], p_k[0], p_k[1])[0]
        #print("{0} - {1}".format(azimuth_ij,azimuth_jk))
        return abs(azimuth_ij - azimuth_jk) 
    
    def more_points_further_than_min_distance(self):
        for self.k in range(self.j+1,self.n):
            p_j = self.traj.to_linestring().coords[int(self.j)]
            p_k = self.traj.to_linestring().coords[int(self.k)]
            d_space_j_k = geod.inv(p_j[0], p_j[1], p_k[0], p_k[1])[2]
            if d_space_j_k >= self.min_distance:
                return True
        return False
        
    def is_representative_max_distance(self):
        p_i = self.traj.to_linestring().coords[int(self.i)]
        p_j = self.traj.to_linestring().coords[int(self.j)]
        #d_space = self.d.measureLine(QgsPoint(p_i.x(),p_i.y()),QgsPoint(p_j.x(),p_j.y()))
        #d_space = math.sqrt( (p_j[0] - p_i[0])**2 + (p_j[1] - p_i[1])**2 ) # sqrt( (x2 - x1)**2 + (y2 - y1)**2 ) revisar igual
        d_space = geod.inv(p_i[0], p_i[1], p_j[0], p_j[1])[2]
        
        if d_space >= self.max_distance:
            self.significant_points.append(p_i)
            return True
        else:
            return False

In [20]:
significant_points = []
for line in trajectories:
    a = Analyzer(traj = line, max_distance = 1000, min_distance = 100, min_stop_duration = 300, min_angle = 45)
    significant_points = significant_points + a.find_significant_points()

In [21]:
print("Number of significant points: {0}".format(len(significant_points)))

Number of significant points: 71


In [22]:
significant_points[1:10]

[(7.218432000000001, 50.75238),
 (7.203697, 50.783398),
 (7.197188000000001, 50.785156),
 (7.205189999999999, 50.79135),
 (7.200433, 50.794785),
 (7.20483, 50.791313),
 (7.19689, 50.785121999999994),
 (7.200432000000001, 50.784508),
 (7.1990419999999995, 50.781673)]

## Group Points in Space